# Assignment 3

*ZM MW*

<font color=magenta>__Game of Life!__</font>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import imageio
import os
import shutil

In [11]:
class GoL:
    
    # possible to specify different alive/dead symbols
    # conditions = [living cell dies if it has less than ... living neighbors, living cell dies if it has more than ... living neighbors, dead cell comes alive if it has ... living neighbors]
    
    def __init__(self, board='game.txt', alive='o', dead='.', conditions=[2,3,3]):
        self.board = pd.read_csv(board, header=None)
        self.board = np.array(list(self.board[0].apply(list)))
        self.replace = {dead:0, alive:1}
        self.board = np.vectorize(self.replace.get)(self.board)
        self.coord = np.array([[-1,-1], [-1,0], [-1,1],
                               [ 0,-1],         [ 0,1],
                               [ 1,-1], [ 1,0], [ 1,1]])
        self.now = self.board
        self.previous_step = np.full(self.board.shape, None)
        self.conditions = conditions
        self.end = False
        
        
    def one_step(self): 
                
        if (~np.all(self.now == self.previous_step)):
            self.previous_step = self.now.copy()
            
            for i in range(self.board.shape[0]):
                for j in range(self.board.shape[1]):  # iterating for all the cells
                    node = self.previous_step[i,j]
                    neighbors = [i,j] + self.coord
                    neighbors = neighbors[(neighbors[:,0] >= 0) & (neighbors[:,1] >= 0) & 
                                               (neighbors[:,0] < self.board.shape[0]) & (neighbors[:,1] < self.board.shape[1]),:]
                            # checking whether neighbors do not exceed the grid (not squared matrices are allowed)
                    living_neighbors = sum(self.previous_step[neighbors.T.tolist()]) # number of living neighbors
                    
                    if node == 1:  # setting new state for a cell
                        if (living_neighbors < self.conditions[0]) | (living_neighbors > self.conditions[1]):
                            self.now[i,j] = 0
                    else:
                        if living_neighbors == self.conditions[2]:
                            self.now[i,j] = 1
            
        else:
            self.end = True   
            

Let's see how the attributes look like in one step.

In [3]:
game = GoL('game1.txt')
game.board.shape

(10, 13)

In [4]:
game.one_step()

In [5]:
game.previous_step

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [6]:
game.now

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Making simulation out of a given .txt initial board.

In [15]:
def animate(my_board, iterations, alive='o', dead='.', conditions=[2,3,3]):
    
    if conditions != [2,3,3]:
        gif_path = my_board.split('.')[0] + "_changed_rules_simulation.gif"
    else:
        gif_path = my_board.split('.')[0] + "_simulation.gif"
        
    folder_path = "game_of_life_frames"
    
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
    os.makedirs(folder_path)
    
    frames_path = folder_path + "/{i}.jpg"
    
    cmap = colors.ListedColormap(['pink', 'k'])

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    ax.set_axis_off()
    ax.grid()
    
    game = GoL(my_board, alive=alive, dead=dead, conditions=conditions)
    im = ax.imshow(game.now, cmap=cmap)
    ax.set_title('Game of Life experiment on a {x}x{y} grid'.format(x=game.board.shape[0], y=game.board.shape[1]))
    plt.savefig(frames_path.format(i=0))
    
    n = 1
    while (game.end == False) & (n < iterations) :
        game.one_step()
        im.set_array(game.now)
        plt.savefig(frames_path.format(i=n))
        n += 1

    with imageio.get_writer(gif_path, mode='I') as writer:
        for i in range(n):
            writer.append_data(imageio.imread(frames_path.format(i=i)))


In [16]:
animate('game.txt', 700)

In [16]:
animate('game1.txt', 40)

In [21]:
animate('game2.txt', 100, alive='x', dead='.')

In [23]:
animate('game3.txt', 100)

In [24]:
animate('game4.txt', 100)

In [10]:
animate('game5.txt', 400)

In [17]:
animate('game1.txt', 40, conditions=[3,6,4])

In [18]:
animate('game5.txt', 400, conditions=[3,6,4])